In [ ]:
file: index.html 
Aggiungi lo script evidenziato sostituendo la tua api-key

<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8" />
    <title>Map</title>
    <base href="/" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <link rel="icon" type="image/x-icon" href="favicon.ico" />

   <script src="https://maps.googleapis.com/maps/api/js?key=YOUR_API_KEY"></script>
 </head>
  <body>
    <app-root></app-root>
  </body>
</html>



In [ ]:
file: app.module.ts

import { NgModule } from '@angular/core';
import { BrowserModule } from '@angular/platform-browser';
import { GoogleMapsModule } from '@angular/google-maps' // Importa questo modulo
import { AppComponent } from './app.component';

@NgModule({
  declarations: [
    AppComponent
  ],
  imports: [
    BrowserModule,
    GoogleMapsModule // Importa questo modulo
  ],
  providers: [],
  bootstrap: [AppComponent]
})
export class AppModule { }


file: app.component.ts

import { Component } from '@angular/core';

@Component({
  selector: 'app-root',
  templateUrl: './app.component.html',
  styleUrls: ['./app.component.css']
})
export class AppComponent {
  title = 'server mappe';
  //Aggiungiamo latitudine e longitudine di un luogo
  center : any;
  position : any;
  label :string;
  constructor()
  {
    this.center={lat: 45.506738, lng: 9.190766};
    this.position = this.center;
    this.label = "ciao";
  }

}


file:app.component.html

{{title}}
<!--Centriamo la mappa alle nostre coordinate-->
<google-map [center]="center">  
  <!--Mettiamo un marker alle nostre coordinate-->
  <map-marker [position]="position" [label]="label" [title]="title">
  </map-marker>
</google-map>



Display Objects

In [ ]:
modifica come segue il file app.component.html

{{title}}
<!--Centriamo la mappa alle nostre coordinate-->
<google-map [center]="center">  
  <!--Mettiamo un marker alle nostre coordinate-->
  <map-marker [position]="position" [label]="label" [title]="title">
  </map-marker>

  <map-circle [center]="position"
              [radius]="50"
              [options] = 'circleOptions'>
  </map-circle>
</google-map>

Aggiungi la proprietà circleOptions al file app.component.ts

this.circleOptions = {fillColor : 'red'}



In [ ]:
nel file app.component.ts aggiungi una proprietà 
markerOptions: google.maps.MarkerOptions;

e nel costruttore inizializzala come segue:
 let iconData : google.maps.Icon = {
      url:'./assets/img/cat_acrobat.ico',
      scaledSize : new google.maps.Size(60,60)
 }
  
this.markerOptions = {icon:iconData}

nel file app.component.html
Modifica i parametri passati al componente Marker come segue
 <map-marker [position]="position" 
              [label]="label" 
              [title]="title"
              [options] = "markerOptions">
 </map-marker>



In [ ]:
aggiungi al file app.component.ts una proprietà
vertices: google.maps.LatLngLiteral[];


nel costruttore assegna i seguenti valori
 this.vertices = [
      {  lat: this.center.lat + 0.001, lng: this.center.lng - 0.002 },
      {  lat: this.center.lat, lng: this.center.lng },
      {  lat: this.center.lat - 0.001, lng: this.center.lng - 0.002}
 ];


aggiungi al file app.component.html (sempre dentro al tag google-map)
<map-polygon [paths]="vertices">
</map-polygon>



GeoJson

In [ ]:
Per gestire i GeoJson in angular dovremo creare un modello che segue la struttura del formato geojson.

export class Geometry {
    type: string;
    coordinates: any; 
    //non sappiamo se coordinates sarà un number[] (Point), number[][] (LineString) o    
    //number [][][] (Polygon) allora usiamo il tipo any: indefinito
}
export class GeoJson {
        public type: string;
        public geometry: Geometry;
        public properties?: any //le proprietà possono variare per ora le lasciamo di tipo 
                                //any
}



In [ ]:
Quello che ci interessa davvero è visualizzare molte zone catastali quindi un vettore di oggetti GeoJson.
export class GeoFeatureCollection
{
    public type: string;  //Nel nostro caso il tipo sarà FeatureCollection
    public features : GeoJson[];
}



In [ ]:
modifica il file app.component.ts come segue

import { AfterViewInit } from '@angular/core';
import { Component, ViewChild } from '@angular/core';
import { GoogleMap } from '@angular/google-maps'
import { GEOJSON, GeoFeatureCollection } from './models/geojson.model';

@Component({
  selector: 'app-root',
  templateUrl: './app.component.html',
  styleUrls: ['./app.component.css']
})
export class AppComponent implements AfterViewInit {
  title = 'server mappe';
  //Aggiungiamo latitudine e longitudine di un luogo
  center: google.maps.LatLngLiteral = { lat: 45.506738, lng: 9.190766 };
  geoJsonObject : GeoFeatureCollection;
  fillColor: string = "#FF0000";  //Colore delle zone catastali
 
  markerList : google.maps.MarkerOptions[];
  zoom = 8;

  constructor() {

    //Questi dati dovremmo scaricarli dal server, per ora li abbiamo copiati nel file     gojson.model.ts
    this.geoJsonObject = GEOJSON;  
    console.log(this.geoJsonObject); //stampo l'oggetto geoJsonObject sulla console
  }
}



In [ ]:
modifica il file app.component.html

<h1>Angular Google Maps Demo</h1>
<google-map  #mapRef [center]="center" [zoom]="zoom" [width]="1000">  
</google-map>


Modifica il file app.component.ts aggiungendo il seguente codice
 @ViewChild('mapRef') mapRef: GoogleMap;
  ngAfterViewInit() {
    this.mapRef.data.addGeoJson(this.geoJsonObject);
  }



In pratica la variabile  #mapRef nel file html crea un riferimento al componente GoogleMap.
<google-map  #mapRef [center]="center" [zoom]="zoom" [width]="1000">  
</google-map>

nel file .ts
@ViewChild('mapRef') mapRef: GoogleMap;  <- Ottiene l’oggetto GoogleMap
  ngAfterViewInit() { <- Quando è stata inizializzata l’interfaccia
    this.mapRef.data.addGeoJson(this.geoJsonObject); <- Aggiungo l’oggetto geoJson alla mappa
  }


In [ ]:
Crea la funzione styleFunc  nel file .ts e aggiungila al metodo this.mapRef.data.setStyle

 @ViewChild('mapRef') mapRef: GoogleMap;
  ngAfterViewInit() {
    this.mapRef.data.addGeoJson(this.geoJsonObject);
    this.mapRef.data.setStyle(this.styleFunc)
  }

styleFunc = (feature: any) =>{
    console.log(feature)
    return ({
      clickable: false,
      fillColor: this.fillColor,
      strokeWeight: 1
    });
  }



In [ ]:
Aggiustiamo la funzione come segue
styleFunc = (feature:any) =>{
    console.log(feature.i.id)
    let newColor = "#FF0000"; //RED
    if(feature.i.id == 0) newColor = "#00FF00"; //GREEN
    else newColor = "#0000FF"; //BLUE
    return ({
      clickable: false,
      fillColor: newColor,
      strokeWeight: 1
    });
  }



Visualizzare Marker

In [ ]:
aggiungi al file app.component.ts il seguente codice che riempie il vettore markerList con 2 marker
markerGenerator()
  {
    this.markerList =[
      {
        position : {
          lng : this.geoJsonObject.features[0].geometry.coordinates[0][0][0],
          lat : this.geoJsonObject.features[0].geometry.coordinates[0][0][1]
        },
        label: String(this.geoJsonObject.features[0].properties.id),
      },
      {
        position : {
          lng : this.geoJsonObject.features[1].geometry.coordinates[0][0][0],
          lat : this.geoJsonObject.features[1].geometry.coordinates[0][0][1]
        },
        label: String(this.geoJsonObject.features[1].properties.id),
      },
    ]

  }


e al file html

<map-marker *ngFor="let markerOption of markerList" [options] = "markerOption">
</map-marker>

